In [2]:
import pandas as pd
import re

import past_guest_module as pm

import sys
sys.path.append('../zoho_data_app/')

import main_module as m

%load_ext autoreload
%autoreload 2


In [3]:
path = r'C:\Users\fajar.fatoni\Documents\Python\Data\check_country_code_analytic.xlsx'
df_analytic = pd.read_excel(path, dtype='string', sheet_name='Sheet1')
count_data_awal = df_analytic.shape[0]
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK070681,Karma Resorts,Other,Past Guests,Karma Kandara,PF305536,Mr,Vincent Laval Joseph,Bontemps,Male,...,2025-03-12 00:00:00,4,HOUSE_USE,V_OWNER_NRB,COMP,4,0,0,Email and Phone,NO
1,KK070935,Karma Resorts,Other,Past Guests,Karma Kandara,PF306086,Mr,Simone,Evans,Female,...,2025-03-09 00:00:00,1,HOUSE_USE,V_OWNER_NRB,COMP,5,0,0,Email and Phone,NO
2,KK064840,Karma Resorts,Other,Past Guests,Karma Kandara,PF294479,Miss,Jeynelle,Mclachlan,Female,...,2025-03-12 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
3,KK066101,Karma Resorts,Other,Past Guests,Karma Kandara,PF294479,Miss,Jeynelle,Mclachlan,Female,...,2025-03-12 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
4,KK066703,Karma Resorts,Other,Past Guests,Karma Kandara,PF294479,Miss,Jeynelle,Mclachlan,Female,...,2025-03-12 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,KB045143,Karma Resorts,Other,Past Guests,Karma Bavaria,<NA>,<NA>,Ramiro,Saavedra,Unknown,...,2025-03-12 00:00:00,30,COMP,STAFF,COMP,1,0,0,Email Only,NO
153,KB047215,Karma Resorts,Other,Past Guests,Karma Bavaria,<NA>,<NA>,De- Allianz Global Corporate & Speciality,Se,Unknown,...,2025-03-11 18:00:00,0,CB,TA,RETAIL,1,0,0,Email Only,NO
154,KB047216,Karma Resorts,Other,Past Guests,Karma Bavaria,<NA>,<NA>,De- Allianz Global Corporate & Speciality,Se,Unknown,...,2025-03-12 18:00:00,0,CB,TA,RETAIL,1,0,0,Email Only,NO
155,KB047217,Karma Resorts,Other,Past Guests,Karma Bavaria,<NA>,<NA>,De- Allianz Global Corporate & Speciality,Se,Unknown,...,2025-03-13 12:00:00,0,CB,TA,RETAIL,1,0,0,Email Only,NO


In [4]:
df_analytic.columns

Index(['Booking_Id', 'Lead Brand', 'Lead Sub- Brand', 'Lead Source',
       'Resort_Name', 'ProfileRef', 'Salutation', 'First_Name', 'Surname',
       'Gender', 'Language', 'DOB', 'Street', 'Area', 'Town', 'County',
       'Country', 'Nationality', 'PostCode', 'TelNo', 'MobileNo', 'Email',
       'Exclude From Mailing', 'Opt In', 'Booking Status', 'DateArrival',
       'DateDepart', 'No. of Nights', 'Market_Segment', 'Media_Code',
       'Media Code Group', 'No. of Adult', 'No. of Children', 'No. of Infants',
       'Contact Type', 'Odyssey Members'],
      dtype='object')

In [5]:
df_analytic.fillna('empty', inplace=True)

In [6]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


In [7]:
m.clean_space(df_email, 'Email')
df_email['Email'] = m.lowercase(df_email, 'Email')

c:\Users\fajar.fatoni\Documents\Python\Task Offline\past_guest_toni_app\../zoho_data_app\main_module.py:94: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)


In [8]:
# hitung jumlah duplicate
count_duplicate_general = df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])].shape[0]

df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])]

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
3,KK066101,Karma Resorts,Other,Past Guests,Karma Kandara,PF294479,Miss,Jeynelle,Mclachlan,Female,...,2025-03-12 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
4,KK066703,Karma Resorts,Other,Past Guests,Karma Kandara,PF294479,Miss,Jeynelle,Mclachlan,Female,...,2025-03-12 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
6,KK059192,Karma Resorts,Other,Past Guests,Karma Kandara,PF281265,Mr,Adrian,Di Lallo,Male,...,2025-03-09 00:00:00,3,DIR,CHAT,DIGITAL,5,2,0,Email and Phone,NO
13,KSH022734,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF302018,Mr,Chris,Lee,Male,...,2025-03-15 00:00:00,2,COMP,STAFF,COMP,1,0,0,Email Only,NO
14,KLM008186,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF302018,Mr,Chris,Lee,Male,...,2025-03-18 00:00:00,2,COMP,STAFF,COMP,1,0,0,Email Only,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,KB045143,Karma Resorts,Other,Past Guests,Karma Bavaria,empty,empty,Ramiro,Saavedra,Unknown,...,2025-03-12 00:00:00,30,COMP,STAFF,COMP,1,0,0,Email Only,NO
153,KB047215,Karma Resorts,Other,Past Guests,Karma Bavaria,empty,empty,De- Allianz Global Corporate & Speciality,Se,Unknown,...,2025-03-11 18:00:00,0,CB,TA,RETAIL,1,0,0,Email Only,NO
154,KB047216,Karma Resorts,Other,Past Guests,Karma Bavaria,empty,empty,De- Allianz Global Corporate & Speciality,Se,Unknown,...,2025-03-12 18:00:00,0,CB,TA,RETAIL,1,0,0,Email Only,NO
155,KB047217,Karma Resorts,Other,Past Guests,Karma Bavaria,empty,empty,De- Allianz Global Corporate & Speciality,Se,Unknown,...,2025-03-13 12:00:00,0,CB,TA,RETAIL,1,0,0,Email Only,NO


In [9]:
df_analytic.drop_duplicates(subset=['TelNo','MobileNo','Email'], inplace=True)
df_analytic.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = df_analytic.shape[0]

df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK070681,Karma Resorts,Other,Past Guests,Karma Kandara,PF305536,Mr,Vincent Laval Joseph,Bontemps,Male,...,2025-03-12 00:00:00,4,HOUSE_USE,V_OWNER_NRB,COMP,4,0,0,Email and Phone,NO
1,KK070935,Karma Resorts,Other,Past Guests,Karma Kandara,PF306086,Mr,Simone,Evans,Female,...,2025-03-09 00:00:00,1,HOUSE_USE,V_OWNER_NRB,COMP,5,0,0,Email and Phone,NO
2,KK064840,Karma Resorts,Other,Past Guests,Karma Kandara,PF294479,Miss,Jeynelle,Mclachlan,Female,...,2025-03-12 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
3,KK058778,Karma Resorts,Other,Past Guests,Karma Kandara,PF281265,Mr,Adrian,Di Lallo,Male,...,2025-03-09 00:00:00,3,DIGITAL,WEBSITE,DIGITAL,5,2,0,Email and Phone,NO
4,KK070894,Karma Resorts,Other,Past Guests,Karma Kandara,PF140764,Mr,Anthony,Smith,Male,...,2025-03-18 00:00:00,17,HOUSE_USE,V_OWNER_NRB,COMP,1,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,KSH023121,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF237937,Mr,Gary,Costello,Unknown,...,2025-03-19 00:00:00,1,COMP,STAFF,COMP,1,0,0,Email and Phone,NO
89,KSH023589,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF307013,Mr,Mark,Tipster,Male,...,2025-03-13 00:00:00,2,COMP,STAFF,COMP,2,0,0,Email and Phone,NO
90,KSH021420,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF294624,Ms,Diana,Moulton,Female,...,2025-03-23 00:00:00,2,COMP,CB,GROUP,2,0,0,Email Only,NO
91,KK066508,Karma Resorts,Other,Past Guests,Karma Kandara,PF297550,Mr.,Mahendra,Kakadia,Male,...,2025-03-10 00:00:00,2,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO


In [10]:
find_member_booking = df_analytic.copy()

find_member_booking.loc[find_member_booking['TelNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['MobileNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'
find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

find_member_booking.replace('empty', '', inplace=True)

In [11]:
find_member_booking.to_csv(r'C:\Users\fajar.fatoni\Documents\Python\Data\cek_member_zoho_analytic.csv', index=False)

In [12]:
count_dedup_member = df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])].shape[0]


df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])]


,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
4,KK070894,Karma Resorts,Other,Past Guests,Karma Kandara,PF140764,Mr,Anthony,Smith,Male,...,2025-03-18 00:00:00,17,HOUSE_USE,V_OWNER_NRB,COMP,1,0,0,Email and Phone,NO
27,KB047813,Karma Resorts,Other,Past Guests,Karma Bavaria,PF264770,Herr,Timotheus Matthaus,Stroebel,Male,...,2025-03-09 00:00:00,1,COMP,GER_OWE,COMP,2,0,2,Email and Phone,NO
37,KB044319,Karma Resorts,Other,Past Guests,Karma Bavaria,PF168470,Herr,Joachim,Schug,Male,...,2025-03-17 00:00:00,3,COMP,GER_OWE,COMP,1,0,0,Email and Phone,NO
47,KB044618,Karma Resorts,Other,Past Guests,Karma Bavaria,PF065347,Dr.,Marion,Peckmann,Female,...,2025-03-09 00:00:00,4,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
54,KB048427,Karma Resorts,Other,Past Guests,Karma Bavaria,PF299103,Frau,Caroline,Impler,Female,...,2025-03-09 00:00:00,2,COMP,GER_OWE,COMP,3,0,0,Email Only,NO
55,KB045754,Karma Resorts,Other,Past Guests,Karma Bavaria,PF147772,Frau,Claudia,Goebl,Female,...,2025-03-09 00:00:00,4,COMP,GER_OWE,COMP,3,0,0,Email Only,NO
56,KB046065,Karma Resorts,Other,Past Guests,Karma Bavaria,PF192683,Mrs,Kathrin,Haas,Female,...,2025-03-09 00:00:00,2,COMP,GER_OWE,COMP,2,0,0,Email Only,NO
60,KK066699,Karma Resorts,Other,Past Guests,Karma Kandara,PF295577,empty,Sourab,Pramanik,Unknown,...,2025-03-17 00:00:00,2,COMP,RESERVED,KC,1,0,0,Email and Phone,NO
86,KSH022025,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF298471,Mr,Raghbir,Chand,Male,...,2025-03-17 00:00:00,1,DIR,GHPA,DIGITAL,2,0,0,Email and Phone,NO


In [13]:
df_analytic.drop(df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])
].index, inplace=True)

df_analytic.reset_index(drop=True, inplace=True)

count_after_dedup_member = df_analytic.shape[0]

In [14]:
count_data_akhir = df_analytic.shape[0]

In [15]:
df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB048322,Karma Resorts,Other,Past Guests,Karma Bavaria,PF305195,Mr,Michael,Vaneekeren,Male,...,2025-03-09 00:00:00,2,DIGITAL,WEBSITE,DIGITAL,2,0,0,Email and Phone,NO
1,KK070935,Karma Resorts,Other,Past Guests,Karma Kandara,PF306086,Mr,Simone,Evans,Female,...,2025-03-09 00:00:00,1,HOUSE_USE,V_OWNER_NRB,COMP,5,0,0,Email and Phone,NO
2,KB048526,Karma Resorts,Other,Past Guests,Karma Bavaria,PF306036,Herr,Joern,Zacher,Unknown,...,2025-03-09 00:00:00,7,OTA,OTA,RETAIL,1,0,0,Email and Phone,NO
3,KK058778,Karma Resorts,Other,Past Guests,Karma Kandara,PF281265,Mr,Adrian,Di Lallo,Male,...,2025-03-09 00:00:00,3,DIGITAL,WEBSITE,DIGITAL,5,2,0,Email and Phone,NO
4,KK069221,Karma Resorts,Other,Past Guests,Karma Kandara,PF302634,Mrs,Erin,Husband,Female,...,2025-03-09 00:00:00,1,HOUSE_USE,V_OWNER_NRB,COMP,6,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,KB047243,Karma Resorts,Other,Past Guests,Karma Bavaria,PF300490,empty,Stephanie,Galli,Unknown,...,2025-03-23 00:00:00,1,OTA,OTA,RETAIL,3,0,0,Email and Phone,NO
80,KB047237,Karma Resorts,Other,Past Guests,Karma Bavaria,PF300464,empty,Teresa,Kronawitter,Unknown,...,2025-03-23 00:00:00,2,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
81,KB047880,Karma Resorts,Other,Past Guests,Karma Bavaria,PF303278,empty,Elena,Endstrasser,Unknown,...,2025-03-23 00:00:00,1,OTA,OTA,RETAIL,3,1,0,Email and Phone,NO
82,KB045457,Karma Resorts,Other,Past Guests,Karma Bavaria,PF291550,empty,Marijn,Goemans,Unknown,...,2025-03-23 00:00:00,1,OTA,OTA,RETAIL,6,0,0,Email and Phone,NO


In [16]:
df_analytic['DOB'].loc[df_analytic['DOB']=='']

Series([], Name: DOB, dtype: string)

In [17]:
df_analytic['DOB'].replace('empty', '', inplace=True)
dob_con_date = pd.to_datetime(df_analytic['DOB'], errors='coerce')
dob_date = dob_con_date.dt.strftime('%Y-%m-%d')

arr_con_date = pd.to_datetime(df_analytic['DateArrival'])
arr_date = arr_con_date.dt.strftime('%Y-%m-%d')

dep_con_date = pd.to_datetime(df_analytic['DateDepart'])
dep_date = dep_con_date.dt.strftime('%Y-%m-%d')

df_analytic['DOB'] = dob_date
df_analytic['DateArrival'] = arr_date
df_analytic['DateDepart'] = dep_date


C:\Users\fajar.fatoni\AppData\Local\Temp\ipykernel_15928\1513714307.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_analytic['DOB'].replace('empty', '', inplace=True)


In [18]:
df_analytic.replace('empty', '', inplace=True)

df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB048322,Karma Resorts,Other,Past Guests,Karma Bavaria,PF305195,Mr,Michael,Vaneekeren,Male,...,2025-03-09,2,DIGITAL,WEBSITE,DIGITAL,2,0,0,Email and Phone,NO
1,KB048680,Karma Resorts,Other,Past Guests,Karma Bavaria,PF306613,Frau,Anneli,Gebel,Female,...,2025-03-09,1,DIGITAL,TEL,RETAIL,3,0,0,Email and Phone,NO
2,KK071192,Karma Resorts,Other,Past Guests,Karma Kandara,PF306559,Mr,Adrien Michel,Marie,Male,...,2025-03-09,1,COMP,GSTAFF,COMP,2,0,0,Email and Phone,NO
3,KB045882,Karma Resorts,Other,Past Guests,Karma Bavaria,PF294032,Frau,Katrin,Zacher,Female,...,2025-03-09,7,DIR,WEBSITE,DIGITAL,2,0,0,Email and Phone,NO
4,KK060680,Karma Resorts,Other,Past Guests,Karma Kandara,PF284764,Mr,Teddy,Hermanto,Male,...,2025-03-09,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,KB045457,Karma Resorts,Other,Past Guests,Karma Bavaria,PF291550,,Marijn,Goemans,Unknown,...,2025-03-23,1,OTA,OTA,RETAIL,6,0,0,Email and Phone,NO
80,KB047243,Karma Resorts,Other,Past Guests,Karma Bavaria,PF300490,,Stephanie,Galli,Unknown,...,2025-03-23,1,OTA,OTA,RETAIL,3,0,0,Email and Phone,NO
81,KB047237,Karma Resorts,Other,Past Guests,Karma Bavaria,PF300464,,Teresa,Kronawitter,Unknown,...,2025-03-23,2,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
82,KB047880,Karma Resorts,Other,Past Guests,Karma Bavaria,PF303278,,Elena,Endstrasser,Unknown,...,2025-03-23,1,OTA,OTA,RETAIL,3,1,0,Email and Phone,NO


In [19]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Data Setelah Script Past Guest Sebelumnya : "+str(count_data_awal))
print("Jumlah Data Yang Duplicate : "+str(count_duplicate_general))
print("Jumlah Data Setelah Hapus Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Data Yang Termasuk Member: "+str(count_dedup_member))
print("Jumlah Data Setelah Hapus Member : "+str(count_after_dedup_member))
print("Jumlah Data Akhir : "+str(count_data_akhir))

Summary
Jumlah Data Setelah Script Past Guest Sebelumnya : 157
Jumlah Data Yang Duplicate : 64
Jumlah Data Setelah Hapus Duplicate : 93
Jumlah Data Yang Termasuk Member: 9
Jumlah Data Setelah Hapus Member : 84
Jumlah Data Akhir : 84


In [20]:
df_analytic.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\final_analytics.xlsx', index=False)